# Linear and convolutional autoencoders

An autoencoder is a neural network that can be used to encode and decode data. The general structure of an autoencoder is shown in the figure below. It consists of two parts: an encoder and a decoder. The encoder compresses the input data into a lower dimensional representation (often referred to as the *latent space representation*) by extracting the most salient features of the data, while the decoder reconstructs the input data from the compressed representation. Therefore, autoencoder is often used for *dimensionality reduction*. In this tutorial, our goal is to compare the performance of two types of autoencoders, a linear autoencoder and a convolutional autoencoder, on reconstructing the [`Fashion-MNIST`](https://github.com/zalandoresearch/fashion-mnist) images. With the help of Covalent, we will see how to break a complex workflow into smaller and more manageable tasks, which allows users to track the task dependencies and execution results of individual steps. Another advantage of Covalent is its ability to auto-parallelize the execution of subtasks.

<div align="center">
<img src="././autoencoder_images/schematic.png" style="width: 40%; height: 40%"/>
</div>

## Building the autoencoders

We will build the two types of autoencoders in [PyTorch](https://pytorch.org/). The linear autoencoder is built on the [`Linear`](https://pytorch.org/docs/stable/generated/torch.nn.Linear.html#torch.nn.Linear) layers, while the convolutional autoencoder is built on the [`Conv2d`](https://pytorch.org/docs/stable/nn.html#torch.nn.Conv2d) layers. Let us first install all the necessary dependencies for this tutorial.

In [2]:
# !pip install cova
# !pip install covalent-dask-plugin
# !pip install torch torchvision
# !pip install matplotlib

We can then start the Covalent UI and the local dispatcher server by running `!covalent start`. The UI will be available at http://localhost:8080. Next, we import the following modules:

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
from torchvision import datasets, transforms
import covalent as ct
from covalent.executor import DaskExecutor
from dask.distributed import LocalCluster

cluster = LocalCluster(processes=True)
dask_executor = DaskExecutor(scheduler_address=cluster.scheduler_address)